# Remarques:

Le notebook ne runnait pas quand j'ai pull
C'est pratique si chacun peut 'Restart the Kernel' avant de push comme ca le code compile pour ceux d'après

# TODO LIST:

quick summary of what is said about the data on SNAP


# Data loading & cleaning

#### Imports

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

#### Dataset description

#TODO: quick summary of what is said about the data on SNAP


#### Features description

   E: election outcome, succesful (1) or not (0)
   
   T: time election was closed
   
   U: candidate id, the one being considered for promotion
   
   N: nominator id
   
   V: we have 3 interesting features in this line
   
      result of the vote (1:support, 0:neutral, -1:oppose)
      
      voter id
      
      time of the vote (yyyy-mm-dd)


#### Data loading

In [26]:
DATA_PATH = "data/"

# create a variable to store all the features
wiki = []

# we use encoding="latin-1" as some user names contain non utf-8 characters
# note that we are not interested in the user names, so we don't care if their names is not properly read
with open(DATA_PATH+"wikiElec.ElecBs3.txt", encoding="latin-1") as f:
    
    # lines of the files
    for line in f.readlines():
        
        # lines are separated with tab: \t
        line = line.split("\t")
        first = line[0]
            
        # election outcome
        if first=="E":
            elec_outcome = line[1][:-1] #[:-1] to remove the \n
            continue
        
        # time the election was closed
        if first=="T":
            elec_time = pd.to_datetime(line[1])
            continue
            
        # candidate id  
        if first=="U":
            candidate_id = line[1]
            continue 
            
        # nominator id
        if first=="N":
            nominator_id = line[1]
            continue
        
        if first=="V":
            vote = line[1]
            voter_id = line[2]
            vote_time = pd.to_datetime(line[3])
            
            #store the results
            wiki.append([elec_outcome, elec_time, candidate_id, voter_id, vote, vote_time, nominator_id])
            continue

In [27]:
# store the result wiki in an appropriate pandas dataframe
wikipedia = pd.DataFrame(wiki, columns = ["elec_outcome", "elec_time", "candidate_id", "voter_id", "vote", "vote_time", "nominator_id"])
wikipedia.head()

,elec_outcome,elec_time,candidate_id,voter_id,vote,vote_time,nominator_id
0,1,2004-09-21 01:15:53,30,3,1,2004-09-14 16:26:00,32
1,1,2004-09-21 01:15:53,30,25,-1,2004-09-14 16:53:00,32
2,1,2004-09-21 01:15:53,30,4,1,2004-09-14 17:08:00,32
3,1,2004-09-21 01:15:53,30,5,1,2004-09-14 17:37:00,32
4,1,2004-09-21 01:15:53,30,6,1,2004-09-14 19:28:00,32


#### Data cleaning

We would not want to begin our analysis using inexisting dates ! Let us check that all dates present in our dataframe are coherent. 

Let's check extreme dates.

In [29]:
print("The last election was closed on", max(wikipedia.elec_time),"\nThe first election was closed on", min(wikipedia.elec_time))
print("The last vote was casted on", max(wikipedia.vote_time),"\nThe first vote was casted on", min(wikipedia.vote_time))

The last election was closed on 2008-01-06 02:18:28 
The first election was closed on 2004-09-17 00:41:00
The last vote was casted on 2105-08-31 10:32:44 
The first vote was casted on 2004-03-28 19:12:00


The dataset is defined on the SNAP website as containing elections & votes until January 2008. 

The time elections were closed seem to be coherent. 
In contrast, there are clearly wrong dates about the time of the votes (we're not in 2105 yet...)

In [31]:
# Let's check how many votes were registered after Jan 2008. 
wikipedia[(wikipedia.vote_time >= '2008-02-01')]

,elec_outcome,elec_time,candidate_id,voter_id,vote,vote_time,nominator_id
1143,0,2006-12-29 16:13:03,300,92,1,2086-05-13 01:04:00,-1
1214,0,2007-09-10 02:08:15,302,92,1,2096-05-13 01:04:00,-1
64807,0,2006-11-07 23:12:34,5045,4963,0,2061-11-17 01:04:00,-1
65201,1,2007-05-04 01:47:24,3537,4963,1,2067-11-21 01:04:00,1137
65654,1,2007-05-04 02:02:48,5092,4963,1,2063-01-22 01:04:00,4476
66494,1,2007-05-04 02:46:43,5144,4963,1,2070-12-02 01:04:00,3453
66593,1,2007-05-04 02:56:08,4547,4963,1,2074-11-29 01:04:00,5058
67665,1,2007-05-04 03:31:09,3691,4963,1,2065-09-17 01:04:00,3691
67856,0,2007-12-10 10:57:36,3455,4963,1,2064-05-21 01:04:00,-1
68512,0,2007-05-04 03:57:13,4468,4963,0,2061-07-28 01:04:00,-1


We will thus consider all votes that are registered after Jan 2008 as parasite and delete them from our dataset.

In [32]:
# Let's clean it up
wikipedia = wikipedia[(wikipedia.vote_time < '2008-02-01')]

In [33]:
# The site is not precise as when exactly in January the votes stopped to be casted. 
# Let's also investigate that in order to get a precise idea of the timelapse we are working on.
wikipedia.vote_time.sort_values(ascending=False)

114017   2008-01-06 02:16:00
114016   2008-01-06 02:10:00
113850   2008-01-06 02:04:00
114015   2008-01-06 01:53:00
113904   2008-01-06 01:12:00
                 ...        
2326     2004-03-28 19:55:00
2325     2004-03-28 19:25:00
2324     2004-03-28 19:22:00
2323     2004-03-28 19:19:00
2322     2004-03-28 19:12:00
Name: vote_time, Length: 114029, dtype: datetime64[ns]

In our dataset, the last votes were casted on January 6th, 2008. 

# Preparing datasets for subsequent analysis  

## Sign_time

In a first part, we will only need to study the sign of the votes (i.e. the vote outcome) and their date (i.e. vote_time). We do not need the ids of the voters or candidatate or nominators. Let's create an appropriate dataset for this case. 

In [34]:
sign_time = wikipedia[['vote','vote_time']]

In [35]:
print(sign_time.shape)
sign_time.head()

(114029, 2)


,vote,vote_time
0,1,2004-09-14 16:26:00
1,-1,2004-09-14 16:53:00
2,1,2004-09-14 17:08:00
3,1,2004-09-14 17:37:00
4,1,2004-09-14 19:28:00


## Neutral or not

What role do neutral votes play in this temporal analysis?
We will need a dataframe with neutral votes and the same one without them 


In [36]:
wikipedia_neutral = wikipedia
wikipedia_signed = wikipedia[wikipedia.vote != 0]

# Temporal analysis

## DATE (yyyy-mm-dd)

We will try to detect any election-related patterns based on the moment that the elections were held. 

We will consider the following election-related outcomes:
1. number of elections
2. fraction of successful elections
3. number of votes
4. fractions of positive votes

The x-axis will be the time at which the election was closed.

We will use mainly visualizations under the form of graphs.

In [ ]:
#TODO : adapt the code below
#not done yet

### By month

In [12]:
# sum of the votes
sign_sum = sign_time.groupby(pd.Grouper(key="Time", freq="M")).sum()
print(sign_sum.tail())

fig = plt.figure(figsize=(12,6))
plt.plot(sign_sum)
plt.xlabel('Months')
plt.ylabel('Sum of the votes')
plt.title('Sum of the votes by months - from 2004-09-30 to 2008-01-31');

# Interpretation:
# majority of the votes are postive

NameError: name 'sign_time' is not defined

In [ ]:
# avg of the votes
sign_avg = sign_time.groupby(pd.Grouper(key="Time", freq="M")).mean()
print(sign_avg.tail())

fig = plt.figure(figsize=(12,6))
plt.plot(sign_avg)
plt.xlabel('Months')
plt.ylabel('Avg of the votes')
plt.title('Avg of the votes by months - from 2004-09-30 to 2008-01-31');

# Interpretation: large drop in positive vote in Jan 2007
# Some pattern?

In [ ]:
wikipedia.head()

In [ ]:
#creating an interactive visualization using plotly
import plotly.express as px
#import plotly.offline as pyo

from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import *
init_notebook_mode()
#pyo.init_notebook_mode()
#pyo.init_notebook_mode(connected=True)
fig = px.line(wikipedia, x='Time', y='Sign', title='Time Series with Range Slider and Selectors')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

### By week

In [ ]:
# avg of the votes
sign_avg_w = sign_time.groupby(pd.Grouper(key="Time", freq="W")).mean()
print(sign_avg_w.tail())

fig = plt.figure(figsize=(16,6))
plt.plot(sign_avg_w)
plt.xlabel('Months')
plt.ylabel('Avg of the votes')
plt.title('Avg of the votes by weeks - from 2004-09-30 to 2008-01-31');

# Interpretation: looks like there is a kind of recurrent pattern 

In [ ]:
# subsample: focus on the left part of the graph
sample = wikipedia[(wikipedia.Time >= '2004-09-14') & (wikipedia.Time <= '2005-09-14')]
sample = sample[['Sign','Time']]

sign_avg_w = sample.groupby(pd.Grouper(key="Time", freq="W")).mean()

fig = plt.figure(figsize=(19,6))
plt.plot(sign_avg_w)
plt.xlabel('Month')
plt.ylabel('Avg of the votes')
plt.title('Avg of the votes by weeks - from 2004-09-14 to 2005-09-14');

# seems like people are less postive at the end of the month ??
# looks like the priod is around 40 days... weird

In [ ]:
# subsample: focus on the right part of the graph
sample = wikipedia[(wikipedia.Time >= '2007-01-01') & (wikipedia.Time <= '2008-01-01')]
sample = sample[['Sign','Time']]

sign_avg_w = sample.groupby(pd.Grouper(key="Time", freq="W")).mean()

fig = plt.figure(figsize=(19,6))
plt.plot(sign_avg_w)
plt.xlabel('Month')
plt.ylabel('Avg of the votes')
plt.title('Avg of the votes by weeks - from 2007-01-01 to 2008-01-01');

# seems like people are less postive at the end of the month ??
# looks like the priod is around 40 days... weird

## Duplicates 
We want to investigate the vote distributions for repeated elections, meaning when the same candidate seeks adminship several times. For this to be possible, we have to chack that we indeed have enough duplicates in our dataset in order to conduct a meaningful analysis.

In [ ]:
wikipedia.head()

In [ ]:
wikipedia.shape

In [ ]:
duplicated = wikipedia[ wikipedia[['Voter','Candidate']].duplicated(keep=False) ] # keep=False to keep all duplicates
duplicated
# duplicate in this context = when a voter voted for the same candidate several time
# we have a lot of duplicates : ~7500

In [ ]:
# But on average, how many time did a voter voted for the same candidate?

voter_candidate = wikipedia[['Voter','Candidate']]
# voter_candidate = voter_candidate[voter_candidate.duplicated()]
count_duplicated = voter_candidate.groupby(voter_candidate.columns.tolist(),as_index=False).size()
count_duplicated

In [ ]:
count_duplicated['size'].describe()

# Interpretation
# Most of the voter voted a single time for a given candidate 
# but some voted up to 5 times for the same candidate

In [ ]:
# Histo of the duplicates (1= no duplication)
plt.hist(count_duplicated['size']);

In [ ]:
# Histo of the duplicates 

count_duplicated_tmp = count_duplicated[count_duplicated['size']>1]

plt.hist(count_duplicated_tmp['size']);

# Interpretation:
# More than 3000 people voted at least 2 times for the same candidate